In [ ]:
from omnia_ae.api import AEEnvironment, AEAPI

In [ ]:
from azure.identity import ClientSecretCredential
import os
credential = ClientSecretCredential(
    tenant_id=os.environ['AZURE_TENANT_ID'],
    client_id=os.environ['AZURE_CLIENT_ID_NON_PRODUCTION'],
    client_secret=os.environ['AZURE_CLIENT_SECRET_NON_PRODUCTION']
)

In [ ]:
api = AEAPI(azure_credential=credential, environment=AEEnvironment.Dev())

In [ ]:
api.get_sources()

In [ ]:
api.get_events("AHA", "*", 1, includeOtherFields=False)

In [ ]:
api.get_realtime_subscriptions()

In [ ]:
api.set_realtime_subscription(connectionString="test"
)
#statusCode: 200, with real connection string

In [ ]:
api.create_subscription(facility="NOR")

In [ ]:
api.delete_subscription(facility="NOR")